In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

In [2]:
def create_NN_graph(nNodes,kij):
    
    G = nx.Graph()
    
    elist = []
    for i in range(nNodes-1):
        edge_tuple = (i,i+1,kij)
        elist.append(edge_tuple)
    G.add_weighted_edges_from(elist)
    return G

def create_bottleneck_k_matrix(nNodes,kij):
    
    k = np.zeros((nNodes,nNodes),dtype=float)

    for i in range(nNodes-1):
        for j in range(i+1,nNodes):
            # nearest neighbors are bonded
            if (abs(i-j)<2):
                k[i,j] = kij
            elif i < nNodes//2 and j < nNodes//2:
                k[i,j] = kij
            elif i > nNodes//2 and j > nNodes//2:
                k[i,j] = kij
            # symmetrize
            k[j,i] = k[i,j]
    return k

def k_to_hessian(k):
    hessian = -k
    for i in range(hessian.shape[0]):
        hessian[i,i] = -np.sum(hessian[i,:])
    return hessian

def compute_correlation_from_hessian(hess):
    M = hess.shape[0]
    hess_evals, hess_evecs = np.linalg.eigh(hess)
    # this translates to the following matrix equations
    gamma = np.diag(1.0/hess_evals)
    gamma[0,0] = 0.
    covar_from_hessian = np.dot(hess_evecs,np.dot(gamma,hess_evecs.T))*0.8
    analytic_corr = covar_to_correlation(covar_from_hessian)
    return analytic_corr, covar_from_hessian

def covar_to_correlation(covar):
    M = covar.shape[0]
    corr = np.empty((M,M),dtype=float)
    for i in range(M):
        for j in range(M):
             corr[i,j] = covar[i,j]/(np.sqrt(covar[i,i]*covar[j,j]))
    return corr

In [3]:
def centrality(A,n):
    e, v = np.linalg.eigh(A)
    centrality = np.zeros(A.shape[0])
    for i in range(1,n+1):
        centrality += 1.0/e[-i]*np.dot(A,v[:,-i])
    cmax = np.amax(-centrality)
    cmin = np.amin(-centrality)
    return 2 * (-centrality-cmin)/(cmax-cmin) - 1
    #return -centrality

In [4]:
nearestNeighborGraph = create_NN_graph(9,100.0)

In [5]:
nx.shortest_path(nearestNeighborGraph,source=0,target=8)

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [6]:
nx.to_numpy_matrix(nearestNeighborGraph)

matrix([[  0., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [100.,   0., 100.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0., 100.,   0., 100.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0., 100.,   0., 100.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0., 100.,   0., 100.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 100.,   0., 100.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 100.,   0., 100.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0., 100.,   0., 100.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.]])

In [7]:
k = create_bottleneck_k_matrix(9,100)
print(k)

[[  0. 100. 100. 100.   0.   0.   0.   0.   0.]
 [100.   0. 100. 100.   0.   0.   0.   0.   0.]
 [100. 100.   0. 100.   0.   0.   0.   0.   0.]
 [100. 100. 100.   0. 100.   0.   0.   0.   0.]
 [  0.   0.   0. 100.   0. 100.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0. 100. 100. 100.]
 [  0.   0.   0.   0.   0. 100.   0. 100. 100.]
 [  0.   0.   0.   0.   0. 100. 100.   0. 100.]
 [  0.   0.   0.   0.   0. 100. 100. 100.   0.]]


In [ ]:
bottleneckHessian = k_to_hessian(k)
bottleneckCorr, bottleneckCovar = compute_correlation_from_hessian(bottleneckHessian)
bottleneckCorr

In [ ]:
Ak = np.copy(k)
for i in range(Ak.shape[0]):
    for j in range(Ak.shape[1]):
        Ak[i,j] /= np.sqrt(np.sum(k[:,i])*np.sum(k[:,j]))
Ak

In [ ]:
scaled_bottleneck??????

In [ ]:
cost_from_k = 1.0/k
for i in range(k.shape[0]): cost_from_k[i,i] = 0.0

for i in range(9-2):
    for j in range(i+2,9):
        if k[i,j] > 0:
            scaled_bottleneck[i,j] *= 1.0/k[i,j]
            scaled_bottleneck[j,i] = scaled_bottleneck[i,j]
G_from_k_bottleneck = nx.from_numpy_matrix(cost_from_k)
nx.shortest_path(G_from_k_bottleneck,source=0,target=8,weight='weight')

In [ ]:
bottleneck_k_betweenness = nx.betweenness_centrality(G_from_k_bottleneck,weight='weight')

In [ ]:
centralityK = centrality(Ak,1)

In [ ]:
print(centralityCorr)

In [ ]:
pairDist = np.empty(k.shape,dtype=np.float64)
for i in range(k.shape[0]):
    for j in range(k.shape[1]):
        pairDist[i,j] = abs(i-j)*2.0
pairDist

In [ ]:
ACorr = abs(bottleneckCorr-np.identity(bottleneckCorr.shape[0]))
ACorr_l1 = np.copy(ACorr)
lamb = 1.0
for i in range(ACorr.shape[0]):
    for j in range(ACorr.shape[1]):
        ACorr_l1[i,j] *= np.exp(-pairDist[i,j]/lamb)
centralityCorr = centrality(ACorr,1)
centralityCorr_l1 = centrality(ACorr_l1,1)

In [ ]:
ACorr_l1 = 1.0/ACorr_l1
for i in range(ACorr_l1.shape[0]): ACorr_l1[i,i] = 0.0

In [ ]:
ACorr = 1.0/ACorr
for i in range(ACorr.shape[0]): ACorr[i,i] = 0.0

In [ ]:
G_from_Corr_bottleneck = nx.from_numpy_matrix(ACorr_l1)
print(nx.shortest_path(G_from_Corr_bottleneck,source=0,target=8,weight='weight'))
print(nx.betweenness_centrality(G_from_Corr_bottleneck,weight='weight'))
bottleneck_corr_l1_betweenness = nx.betweenness_centrality(G_from_Corr_bottleneck,weight='weight')

In [ ]:
G_from_Corr_bottleneck = nx.from_numpy_matrix(ACorr)
print(nx.shortest_path(G_from_Corr_bottleneck,source=0,target=8,weight='weight'))
print(nx.betweenness_centrality(G_from_Corr_bottleneck,weight='weight'))
bottleneck_corr_betweenness = nx.betweenness_centrality(G_from_Corr_bottleneck,weight='weight')

In [ ]:
ACorr

In [ ]:
k

In [ ]:
bottleneck_k_betweenness[2]

In [ ]:
plt.plot(*zip(*sorted(bottleneck_k_betweenness.items())),label="k")
plt.plot(*zip(*sorted(bottleneck_corr_betweenness.items())),label="corr")
plt.plot(*zip(*sorted(bottleneck_corr_l1_betweenness.items())),label="corr $\lambda=1$")
#plt.plot(centralityCorr_l5,label="corr $\lambda=5$")
#plt.plot(centralityCorr_l10,label="corr $\lambda=10$")
#plt.plot(centralityCorr_l15,label="corr $\lambda=15$")
#plt.plot(centralityCorr_l50,label="corr $\lambda=50$")
plt.legend()

In [ ]:
plt.plot(centralityK,label="k")
plt.plot(centralityCorr,label="corr $\lambda\Rightarrow\infty$")
plt.plot(centralityCorr_l1,label="corr $\lambda=1$")
#plt.plot(centralityCorr_l10,label="corr $\lambda=10$")
#plt.plot(centralityCorr_l15,label="corr $\lambda=15$")
#plt.plot(centralityCorr_l50,label="corr $\lambda=50$")
plt.legend()

In [ ]:
# create the hessian matrix for the 9 node nearest neighbor spring system
kij = 303.1
b = 2
k_9  = create_NN_k_matrix(9,kij)
k_9

In [ ]:
k2 = k_9 + np.ones((9,9))
x2 = np.zeros(9)
x2[0] = -kij * b
x2[8] = -x2[0]
print(np.dot(np.linalg.inv(k2),x2))


In [ ]:
# diagonlize the hessian
k_9_evals, k_9_evecs = np.linalg.eigh(k_9)
idx = k_9_evals.argsort()
k_9_evals = k_9_evals[idx]
k_9_evecs = k_9_evecs[:,idx]
#print k_9_evecs
k_9_evals

In [ ]:
# here I use the eq 13 from Chennubhotla et al to compute the covariance directly from eigenvectors 
covar_from_hessian = np.zeros((9,9),dtype=float)
for i in range(9):
    for j in range(9):
        for k in range(1,9):
            covar_from_hessian[i,j] += 1.0/k_9_evals[k] * k_9_evecs[i,k] * k_9_evecs[j,k]
covar_from_hessian *= 0.8
covar_from_hessian

In [ ]:
# this translates to the following matrix equations
gamma = np.diag(1.0/k_9_evals)
gamma[0,0] = 0.
covar_from_hessian = np.dot(k_9_evecs,np.dot(gamma,k_9_evecs.T))*0.8

In [8]:
# read in Heidi's covariance matrix from simulation
covar = np.loadtxt("covariance_nearest_neighbors.dat")

In [9]:
covar

array([[ 0.00668836,  0.00440812,  0.00236705,  0.00049752, -0.00093851,
        -0.0019904 , -0.00308745, -0.00387568, -0.00406901],
       [ 0.        ,  0.00459417,  0.00260601,  0.00074953, -0.00068842,
        -0.00177747, -0.00268291, -0.0035176 , -0.00369142],
       [ 0.        ,  0.        ,  0.00319458,  0.00131588, -0.00014785,
        -0.00125902, -0.00218964, -0.00285816, -0.00302885],
       [ 0.        ,  0.        ,  0.        ,  0.00218494,  0.0007313 ,
        -0.00036917, -0.00122547, -0.00183341, -0.00205113],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.00196295,
         0.00079227, -0.00010053, -0.00068035, -0.00093085],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.00212061,  0.00131675,  0.00077116,  0.00039527],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.00313119,  0.00267069,  0.00216736],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0

In [10]:
# let's try and recompute Heidi's covariance
heidi_traj = np.loadtxt("positions.dat")
nSteps = heidi_traj.shape[0]
nNodes = heidi_traj.shape[1]
covar = np.zeros((nNodes,nNodes),dtype=np.float64)
avg = np.zeros(nNodes,dtype=np.float64)
for step in range(100,nSteps):
    # substract center-of-geometry
    heidi_traj[step,:] -= np.mean(heidi_traj[step,:])
    # add to average
    avg += heidi_traj[step,:]
    # add to covar
    covar += np.dot(heidi_traj[step,:].reshape(nNodes,1),heidi_traj[step,:].reshape(1,nNodes))
# finish averages
avg /= (nSteps-100)
covar /= (nSteps-100)
# finish covar
covar -= np.dot(avg.reshape(nNodes,1),avg.reshape(1,nNodes))
print(covar)

[[ 0.00681189  0.00452911  0.00244236  0.00053086 -0.00091684 -0.00203058
  -0.00316174 -0.00400121 -0.00420384]
 [ 0.00452911  0.00470496  0.00270411  0.00079652 -0.00068009 -0.00184616
  -0.0027641  -0.0036324  -0.00381196]
 [ 0.00244236  0.00270411  0.00320864  0.00132241 -0.00014484 -0.00129211
  -0.00224564 -0.00291952 -0.00307541]
 [ 0.00053086  0.00079652  0.00132241  0.00219356  0.00072314 -0.00039649
  -0.00126565 -0.00186118 -0.00204317]
 [-0.00091684 -0.00068009 -0.00014484  0.00072314  0.00198099  0.00081446
  -0.00011513 -0.00071351 -0.00094819]
 [-0.00203058 -0.00184616 -0.00129211 -0.00039649  0.00081446  0.00216203
   0.00135895  0.00080648  0.00042342]
 [-0.00316174 -0.0027641  -0.00224564 -0.00126565 -0.00011513  0.00135895
   0.00317343  0.00277004  0.00224984]
 [-0.00400121 -0.0036324  -0.00291952 -0.00186118 -0.00071351  0.00080648
   0.00277004  0.00495601  0.0045953 ]
 [-0.00420384 -0.00381196 -0.00307541 -0.00204317 -0.00094819  0.00042342
   0.00224984  0.00459

In [ ]:
covar/covar_from_hessian

In [ ]:
avg_positions = np.mean(heidi_traj[100:1000,:],axis=0)
print(avg_positions)

In [ ]:
400*np.mean(covar/covar_from_hessian)

In [ ]:
def covar_to_correlation(covar):
    M = covar.shape[0]
    corr = np.empty((M,M),dtype=float)
    for i in range(M):
        for j in range(M):
             corr[i,j] = covar[i,j]/(np.sqrt(covar[i,i]*covar[j,j]))
    return corr

In [ ]:
measured_corr = covar_to_correlation(covar)
analytic_corr = covar_to_correlation(covar_from_hessian)

In [ ]:
measured_corr

In [ ]:
analytic_corr

In [ ]:
np.savetxt("9node_nearestneighbor_analytic_corr.dat",analytic_corr)

In [ ]:
def compute_correlation_from_hessian(hess):
    M = hess.shape[0]
    hess_evals, hess_evecs = np.linalg.eig(hess)
    # sort eigenvectors and eigenvalues in ascending order (of eigenvalues)
    idx = hess_evals.argsort()
    hess_evals = hess_evals[idx]
    hess_evecs = hess_evecs[:,idx]
    print(hess_evecs)
    # this translates to the following matrix equations
    gamma = np.diag(1.0/hess_evals)
    gamma[0,0] = 0.
    covar_from_hessian = np.dot(hess_evecs,np.dot(gamma,hess_evecs.T))*0.8
    analytic_corr = covar_to_correlation(covar_from_hessian)
    return analytic_corr, covar_from_hessian

In [ ]:
k9_NN_plus_19 = np.copy(k_9)
k9_NN_plus_19[0,8] = k9_NN_plus_19[8,0] = -kij
k9_NN_plus_19[0,0] += kij
k9_NN_plus_19[8,8] += kij
print(k9_NN_plus_19)

In [ ]:
k9_NN_plus_19_corr, k9_NN_plus_19_covar = compute_correlation_from_hessian(k9_NN_plus_19)

In [ ]:
k9_NN_plus_19_covar

In [ ]:
np.savetxt("9node_nearestneighbor_plus_1_9_analytic_corr.dat",k9_NN_plus_19_corr)

In [ ]:
# read in Heidi's covariance matrix from simulation
covar_19 = np.loadtxt("covariance_9springs.dat")
corr_19 = covar_to_correlation(covar_19)
corr_19

In [ ]:
def linear_MI_rMI_from_covar(covar,d=1):
    # size of the array for MI and rMI is the number of nodes
    N = covar.shape[0]//d
    # declare MI and rMI matrices
    rMI = np.zeros((N,N),dtype=float)
    MI = np.zeros((N,N),dtype=float)
    # check dimensionality of system - 1D we need to avoid taking determinants
    if (d==1):
        # loop over unique pairs of nodes
        for i in range(N-1):
            for j in range(i+1,N):
                # compute numerator in argument of log of linear MI equation
                temp = covar[i,i]*covar[j,j]
                # compute linear MI (eq 10 of Grubmuller 2005)
                MI[i,j] = 0.5*np.log(temp/(temp-covar[i,j]**2))
                # symmetrize MI
                MI[j,i] = MI[i,j]
                # compute rMI (eq 9 from Grubmuller 2005 with sqrt instead of inverse sqrt - typo in paper)
                rMI[i,j] = np.sqrt(1.0-np.exp(-2.0*MI[i,j]/d))
                # symmetrize rMI
                rMI[j,i] = rMI[i,j]
    else:
        # loop over unique pairs of nodes
        for i in range(N-1):
            # i index assuming that each node has d values sequentially populating the covar matrix
            iIndex = i*d
            for j in range(i+1,N):
                # j index assuming that each node has d values sequentially populating the covar matrix
                jIndex = j*d
                # compute numerator in argument of log of linear MI equation
                temp = np.linalg.det(covar[iIndex:iIndex+d,iIndex:iIndex+d])*np.linalg.det(covar[jIndex:jIndex+d,jIndex:jIndex+d])
                # make list of indeces for the 2d X 2d C_ij matrix
                idx = np.append(np.arange(iIndex,iIndex+d,1),np.arange(jIndex,jIndex+d,1))
                # compute linear MI (eq 10 of Grubmuller 2005)
                MI[i,j] = 0.5*np.log(temp/np.linalg.det(covar[np.ix_(idx,idx)]))
                # symmetrize
                MI[j,i] = MI[i,j]
                # compute rMI (eq 9 from Grubmuller 2005 with sqrt instead of inverse sqrt - typo in paper)
                rMI[i,j] = np.sqrt(1.0-np.exp(-2.0*MI[i,j]/d))
                # symmetrize
                rMI[j,i] = rMI[i,j]
    
    # populate diagonal elements of MI and rMI
    MI += np.diag(np.full(N,np.inf))
    rMI += np.diag(np.ones(N))
    # return MI and rMI
    return MI,rMI    

def linear_MI_from_covar(covar):
    M = covar.shape[0]
    MI = np.empty((M,M))
    for i in range(M):
        for j in range(M):
            temp = covar[i,i]*covar[j,j]
            MI[i,j] = 0.5*np.log(temp/(temp-covar[i,j]**2))
    return MI

In [ ]:
k9_NN_plus_19_MI,k9_NN_plus_19_rMI  = linear_rMI_from_covar(k9_NN_plus_19_covar,d=3)
k9_NN_plus_19_rMI
k9_NN_plus_19_MI

In [ ]:
idx = np.arange(0,3,1)
idx = np.append(idx,np.arange(6,9,1))
print idx

In [ ]:
k9_NN_plus_19_covar[np.ix_(idx,idx)].shape

In [ ]:
k9_NN_plus_19_covar[0:3,0:3]

In [ ]:
N=9
np.diag(np.ones(N))

In [ ]:
np.linalg.inv(covar_from_hessian)

In [ ]:
alpha, a = np.linalg.eig(covar_from_hessian/0.8)
idx = alpha.argsort()
alpha = alpha[idx]
a = a[:,idx]
print alpha

In [ ]:
kappa = np.diag(1.0/alpha)
kappa[0,0] = 0.
krev = np.dot(a,np.dot(kappa,a.T))

In [ ]:
print krev

In [ ]:
def k_from_covar(covar):
    alpha, a = np.linalg.eig(covar_from_hessian/0.8)
    idx = alpha.argsort()
    alpha = alpha[idx]
    a = a[:,idx]
    kappa = np.diag(1.0/alpha)
    kappa[0,0] = 0.
    krev = np.dot(a,np.dot(kappa,a.T))
    return krev